# Cephノード構築：OSD用LVの生成
CepnクラスタにOSD用のLVを生成

# パラメータ定義

## パラメータファイル設定

Cephクラスタの定義ファイルを指定するため、<br>
パラメータ定義格納ディレクトリを表示する。

In [ ]:
!ls -p ~/notebooks/share/ | grep "/$"

In [ ]:
!ls -p ~/notebooks/share/configuration

In [ ]:
!ls -p ~/notebooks/share/configuration/ceph

In [ ]:
import sys, os.path, importlib, json, yaml

config_file_name = "ceph_admin_vm.yml"
config_file_path = os.path.expanduser("~/notebooks/share/configuration/ceph/{}".format(config_file_name))
with open(config_file_path) as f:
    admin_params = yaml.safe_load(f)
admin_params

In [ ]:
target_vm_name = 'xxx-ceph-admin1'
admin_patam = admin_params[target_vm_name]
admin_patam

In [ ]:
ceph_admin_ip = admin_patam['network']['provisioning']['ip']

In [ ]:
import sys, os.path, importlib, json, yaml

config_file_name = "xxx-openstack-parameter.yml"
config_file_path = os.path.expanduser("~/notebooks/share/configuration/{}".format(config_file_name))
with open(config_file_path) as f:
    params = yaml.safe_load(f)
params

In [ ]:
repo_server = params['repository']['addresses']['service_operation']
repo_server

In [ ]:
provisioning_vip = params['openstack']['provisioning_vip']
provisioning_vip

In [ ]:
network_segment_controller_vip = params['network_segment_controller_vip']
network_segment_controller_vip

## CephノードとストレージセグメントIPアドレス対応表の読み込み

In [ ]:
import os
import csv
import pandas as pd

instance_ip_csv_path =  os.environ['HOME'] + "/notebooks/share/configuration/ceph/ceph_instance_ip.csv"
instance_ip_list = []
instance_ip_map = {}

with open(instance_ip_csv_path, 'r') as path_csv:
    reader = csv.DictReader(path_csv)
    
    for record in reader:
        instance_ip_list.append(record)
        instance_ip_map[record['instance_name']] = record

pd.DataFrame(instance_ip_list)

## 作業用ディレクトリを作成する

In [ ]:
import os,tempfile
temp_dir = tempfile.mkdtemp()
print (temp_dir)

## AnsibleのInventoryファイルを作業用ディレクトリへ作成する

In [ ]:
import os

with open( os.path.join(temp_dir, "hosts"), 'w') as f:
    f.write('''[openstack_ctl]
{openstack_cmn}
'''.format(openstack_cmn = provisioning_vip))

hosts = temp_dir + "/hosts"
!cat $hosts

In [ ]:
%env ANSIBLE_INVENTORY={hosts}

疎通確認

In [ ]:
!ansible -m ping openstack_ctl

In [ ]:
!ansible -a 'hostname' openstack_ctl

## OpenStack用OpenRCファイル設定

`xxxxxxx`プロジェクト用のOpenRCファイルを事前にダウンロードしておくこと。  

In [ ]:
import os

user_creds = os.path.expanduser('~/.keys/xxxxxxx-openrc.sh')

assert os.path.exists(user_creds), '{} is not exist'.format(user_creds)

In [ ]:
%env USER_CREDS={user_creds}

In [ ]:
!ls -l {user_creds}

## openstackコマンド用ユーティリティ関数

In [ ]:
def build_cmdline(param):
    cmdline = ''
    
    for param_name, param_value in param.items():
        if isinstance(param_value, dict):
            for key, value in param_value.items():
                cmdline += ' --{} {}={}'.format(param_name, key, value)
        elif isinstance(param_value, list):
            for item in param_value:
                cmdline += ' --{} {}'.format(param_name, item)
        elif isinstance(param_value, bool):
            if param_value:
                cmdline += ' --{}'.format(param_name)
        else:
            cmdline += ' --{} {}'.format(param_name, param_value)

    return cmdline

In [ ]:
import json

def run_openstack(cmd):
    print('EXEC: {}'.format(cmd))
    out = !source {user_creds} && {cmd}
    print('OUTPUT: {}'.format('\n'.join(out)))
    return json.loads('\n'.join(out))

テスト

In [ ]:
run_openstack('openstack flavor list -f json' + build_cmdline({'all': True}))

## デプロイされたCephノード情報の取得

ストレージノードを特定するにあたり、ノード名称が　ceph-nodexxx （xxxの部分は、右寄せ0埋めの数字3桁）というルールに従っていることを前提としている。<br>
 例：ceph-node001<br>
 このノード名称ルールは、構築するCephクラスタ毎に変えるため、以下で定義する。

In [ ]:
# 初期化対象のインスタンス名称を設定する
raise Exception('この先手作業が必要です')

In [ ]:
import pandas as pd
import re

# 対象インスタンス名称マッチングルール
name_match_rule = r'ceph-xxx-node-[0-9]+'

## OpenStack Ironicからノード情報の取得

In [ ]:
cmd = "openstack baremetal node list -f json --long"
nodes = run_openstack(cmd)

In [ ]:
import pandas as pd
import re

for node in nodes:
    instance_uuid = node.get("Instance UUID", None)
    if instance_uuid is None:
        continue
        
    node["instance_name"] = node["Instance Info"]["display_name"]

pd.set_option('display.max_rows', None)
pd.DataFrame(nodes, columns=['Instance UUID', 'instance_name', 'UUID', 'Maintenance', 'Provisioning State', 'Power State', 'Name'])


In [ ]:
cmd = "openstack server list -f json --long"
nova_server_map = run_openstack(cmd)

In [ ]:
node_list = []
for server in nova_server_map:
    node_id = server['ID']
    if re.match(name_match_rule, server['Name']) is None:
        continue
    for node in nodes:
        if node['Instance UUID'] == node_id:
            node_info = {}
            node_list.append(node_info)
            node_info['name'] = server['Name']
            node_info['node_name'] = node['Name']
            instance_ip_info = instance_ip_map.get(server['Name'], None)
            if instance_ip_info is not None:
                node_info['instance_ip'] = instance_ip_info.get('ipv4', None)
            else:
                node_info['instance_ip'] = None
            address = server['Networks']
            prov_net = address['provisioning-net']
            for addr in prov_net:
                node_info['prov_ip'] = addr

pd.DataFrame(node_list)

## 初期化対象Cephノード情報の取得

初期化対象のインスタンス名称を設定する

### 対象ノードの指定

In [ ]:
# 初期化対象のインスタンス名称を設定する
raise Exception('この先手作業が必要です')

In [ ]:
# bootstrap OSD生成先ノード
instance_name = 'ceph-xxx-node-002'

In [ ]:
bootstrap_node_name = 'ceph-xxx-node-001'

In [ ]:
target = None
for elem in node_list:
    if elem['name'] == instance_name:
        target = elem
        break

assert (target is not None), "指定されたターゲットノードが存在しません。"
print (target)

In [ ]:
bootstrap_node = None
for elem in node_list:
    if elem['name'] == bootstrap_node_name:
        bootstrap_node = elem
        break

assert (bootstrap_node is not None), "指定されたターゲットノードが存在しません。"
print (bootstrap_node)

## 一時Inventoryファイルの生成
以下を実行し、作業用ディレクトリ配下に一時Inventoryファイルを生成する。

Inventoryファイルを生成する。

In [ ]:
node_user='xxxxx'
proxy_user='xxxxx'
proxy_key='~/.ssh/id_ras'

sudoでパスワードが必要な場合は、パスワード入力を設定してansible.cfgに設定すること。

In [ ]:
import os

with open( os.path.join(temp_dir, "hosts"), 'w') as f:
     f.write('''[ceph_admin]
{ceph_admin_ip}   ansible_user={proxy_user} ansible_ssh_private_key_file={proxy_key} ansible_python_interpreter=/usr/bin/python3
[ceph_node]
{node_ip}  ansible_user={node_user} ansible_ssh_common_args='-o ControlMaster=auto -o StrictHostKeyChecking=no -o ControlPersist=30m -o ProxyCommand="ssh -W %h:%p -i {proxy_key} -q {proxy_user}@{ceph_admin_ip}"' ansible_python_interpreter=/usr/bin/python3
'''.format(ceph_admin_ip=ceph_admin_ip, node_ip=target["prov_ip"],
           node_user=node_user, proxy_user=proxy_user, proxy_key=proxy_key))
     f.write('''[bootstrap]
{node_ip}  ansible_user={node_user} ansible_ssh_common_args='-o ControlMaster=auto -o StrictHostKeyChecking=no -o ControlPersist=30m -o ProxyCommand="ssh -W %h:%p -i {proxy_key} -q {proxy_user}@{ceph_admin_ip}"' ansible_python_interpreter=/usr/bin/python3
'''.format(ceph_admin_ip=ceph_admin_ip, node_ip=bootstrap_node["prov_ip"],
           node_user=node_user, proxy_user=proxy_user, proxy_key=proxy_key))

hosts = temp_dir + "/hosts"

!cat $hosts

In [ ]:
%env ANSIBLE_INVENTORY={hosts}

# 疎通確認
以下コマンドを実行し、OpeHubからCephAdmin経由で、Cephノードに疎通することを確認する。

In [ ]:
!ansible -i $hosts -b -m shell -a 'hostname' ceph_admin

以下コマンドを実行し、OpeHubからCephAdminに疎通することを確認する。

In [ ]:
!ansible -i $hosts -b -m shell -a 'hostname' bootstrap

In [ ]:
!ansible -i $hosts -b -m shell -a 'hostname' ceph_node

# 事前物理Disk情報取得

In [ ]:
ret = !ansible -i $hosts -b -m shell -a 'lsblk -J' ceph_node
ret.pop(0)

json_text = ""
for line in ret:
    json_text = json_text + line

disk_data = json.loads(json_text)

In [ ]:
print(json.dumps(disk_data, sort_keys=True, indent=4))

In [ ]:
def get_children(src):
    ret = []
    children = src.get('children', [])
    for blockdevice in children:
        blk_info = {}
        blk_info['name'] = blockdevice['name']
        blk_info['size'] = blockdevice['size']
        blk_info['type'] = blockdevice['type']
        ret.append(blk_info)
        child = get_children(blockdevice)
        if len(child) > 0:
            blk_info['children'] = child
    return ret
        

In [ ]:
disk_info_list = []
for blockdevice in disk_data['blockdevices']:
    blk_info = {}
    blk_info['name'] = blockdevice['name']
    blk_info['size'] = blockdevice['size']
    blk_info['type'] = blockdevice['type']
    disk_info_list.append(blk_info)
    child = get_children(blockdevice)
    if len(child) > 0:
        blk_info['children'] = child   

print(json.dumps(disk_info_list, sort_keys=True, indent=4))

In [ ]:
db_disk = None
data_disks = []
for disk_info in disk_info_list:
    if disk_info['type'] != 'disk':
        continue
    if 'children' in disk_info:
        if len(disk_info['children']) <= 2:
            db_disk = disk_info
    else:
        data_disks.append(disk_info)

print(db_disk)
print(data_disks)


# パーティション情報取得

In [ ]:
part_info = !ansible -i $hosts -b -m shell -a "parted -mlj 2> /dev/null" ceph_node
part_info.pop(0)
print(part_info)

In [ ]:
import re

part_info_json_list = []
part_info_json = ""
for part_info_line in part_info:
    if part_info_line == '':
        if len(part_info_json) > 0:
            part_info_json_list.append(part_info_json)
        part_info_json = ""
        continue
    part_info_json = part_info_json + part_info_line

part_info_list = []
for part_info_json in part_info_json_list:
    part_info_list.append(json.loads(part_info_json))

print(json.dumps(part_info_list, sort_keys=True, indent=4))

# 事前PV/LVM情報取得

## LVM情報取得関数定

In [ ]:
import re

def make_pv_info_list(ret):
    ret.pop(0)
    pv_info_list = []
    pv_info = None
    for line in ret:
        if re.search('Physical volume', line) is not None:
            pv_info = {}
        elif re.search('PV Name', line) is not None:
            elems = re.sub('PV Name','',line).split()
            pv_info['pv_name'] = elems[0]
        elif re.search('VG Name', line) is not None:
            elems = re.sub('VG Name','',line).split()
            pv_info['vg_name'] = elems[0]
        elif re.search('PV Size', line) is not None:
            elems = re.sub('PV Size','',line).split()
            pv_info['pv_size'] = elems[0]
            pv_info_list.append(pv_info)

    return pv_info_list

def make_lv_info_list(ret, pv_info_map):
    lv_info_list = []
    lv_info = None
    for line in ret:
        if re.search('Logical volume', line) is not None:
            lv_info = {}
        elif re.search('LV Path', line) is not None:
            elems = re.sub('LV Path','',line).split()
            lv_info['lv_path'] = elems[0]
        elif re.search('LV Name', line) is not None:
            elems = re.sub('LV Name','',line).split()
            lv_info['lv_name'] = elems[0]
        elif re.search('VG Name', line) is not None:
            elems = re.sub('VG Name','',line).split()
            lv_info['vg_name'] = elems[0]
            lv_info['pv_name'] = pv_info_map.get(lv_info['vg_name'],None)
        elif re.search('Block device', line) is not None:
            elems = re.sub('Block device','',line).split()
            lv_info['block_dev'] = elems[0]
            dev_elems = lv_info['block_dev'].split(':')
            dev_name = "dm-{}".format(dev_elems[1])
            lv_info['dev_name'] = dev_name
            lv_info_list.append(lv_info)
            
    return lv_info_list

## PV情報取得

In [ ]:
# get PV Info
ret = !ansible -i $hosts -b -m shell -a 'pvdisplay' ceph_node
pv_info_list = make_pv_info_list(ret)

pv_info_map = {}
for pv_info in pv_info_list:
    vg_name = pv_info['vg_name']
    pv_info_map[vg_name] = pv_info['pv_name']    
    
# get PV Info
ret = !ansible -i $hosts -b -m shell -a 'lvdisplay' ceph_node
lv_info_list = make_lv_info_list(ret,pv_info_map)



In [ ]:
pd.DataFrame(pv_info_list)

In [ ]:
pd.DataFrame(lv_info_list)

In [ ]:
db_pv_name = None
data_pv_name_list = []
for pv_info in pv_info_list:
    vg_name = pv_info['vg_name']
    if vg_name == 'vg_db':
        db_pv_name = pv_info['pv_name']
    elif 'vg_data_' in vg_name:
        data_pv_name_list.append(pv_info['pv_name'])
    
print("db_pv_name={}".format(db_pv_name))
print("data_pv_name_list={}".format(data_pv_name_list))


In [ ]:
assert (db_pv_name is None), "すでにOSD用にLVが構築されている"

# OSD構築対象ブロックデバイスの指定
対象ノードのブロックデバイス一覧から、OSDで使用するブロックデバイスを指定する。

## ターゲットディスク情報設定

In [ ]:
ssd_blk = "/dev/" + db_disk["name"]
print("ssd_blk={}".format(ssd_blk))

In [ ]:
ssd_part_info = None
for part_info in part_info_list:
    if part_info['disk']['path'] == ssd_blk:
        ssd_part_info = part_info['disk']
        break

print(ssd_part_info)

In [ ]:
db_disk_partitions = ssd_part_info['partitions']
pd.DataFrame(db_disk_partitions)

In [ ]:
pd.DataFrame(data_disks)

In [ ]:
# OSD接続対象ブロックデバイスを確認する
raise Exception('OSD接続対象ブロックデバイスを確認する')

In [ ]:
# OSDデータ用ブロックデバイスから除外するデバイスを指定

skip_dev_list = []

In [ ]:
block_dev_list = []
for data_disk in data_disks:
    if data_disk['name'] not in skip_dev_list:
        block_dev_list.append(data_disk['name'])

print(block_dev_list)

In [ ]:
print("DataDisk num={}".format(len(block_dev_list)))

# BlueStore DB BlockDevice作成

In [ ]:
ods_vl_map = {}

## 空き領域からLVM用ブロックデバイス生成

In [ ]:
end_part = None
for db_disk_partition in db_disk_partitions:
    if end_part is None:
        end_part = db_disk_partition
    elif end_part['number'] < db_disk_partition['number']:
        end_part = db_disk_partition
        
print(end_part)

In [ ]:
end_pos = end_part['end']
print(end_pos)

In [ ]:
!ansible -i $hosts -b -m shell -a "parted -s -a optimal $ssd_blk -- mkpart primary xfs $end_pos -1" ceph_node

### パーティション情報取得

In [ ]:
part_info = !ansible -i $hosts -b -m shell -a "parted -mlj 2> /dev/null" ceph_node
part_info.pop(0)
print(part_info)

In [ ]:
import re

part_info_json_list = []
part_info_json = ""
for part_info_line in part_info:
    if part_info_line == '':
        if len(part_info_json) > 0:
            part_info_json_list.append(part_info_json)
        part_info_json = ""
        continue
    part_info_json = part_info_json + part_info_line

part_info_list = []
for part_info_json in part_info_json_list:
    part_info_list.append(json.loads(part_info_json))

print(json.dumps(part_info_list, sort_keys=True, indent=4))

In [ ]:
ssd_part_info = None
for part_info in part_info_list:
    if part_info['disk']['path'] == ssd_blk:
        ssd_part_info = part_info['disk']
        break

print(ssd_part_info)

In [ ]:
db_disk_partitions = ssd_part_info['partitions']
pd.DataFrame(db_disk_partitions)

In [ ]:
target_part = None
for db_disk_partition in db_disk_partitions:
    if db_disk_partition['start'] == end_pos:
        target_part = db_disk_partition
        break
        
print(target_part)

In [ ]:
pos = target_part['number']
print(pos)

In [ ]:
!ansible -i $hosts -b -m shell -a "parted -s -a optimal $ssd_blk -- set $pos lvm on" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "parted $ssd_blk print free" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a 'lsblk' ceph_node

## PV/VG生成

In [ ]:
!ansible -i $hosts -b -m shell -a 'pvscan' ceph_node

In [ ]:
pv_bkl = "{}{}".format(ssd_blk,pos)
print(pv_bkl)

In [ ]:
!ansible -i $hosts -b -m shell -a "pvcreate $pv_bkl" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a 'pvscan' ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "pvdisplay $pv_bkl" ceph_node

In [ ]:
ssd_vg_name = "vg_db"

In [ ]:
!ansible -i $hosts -b -m shell -a "vgcreate $ssd_vg_name $pv_bkl" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "vgdisplay" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "pvdisplay $pv_bkl" ceph_node

## DB用LV生成

In [ ]:
osd_num = len(block_dev_list)
print(osd_num)

In [ ]:
import math
lv_size = 1.0 / osd_num * 100.0
lv_size = math.floor(lv_size)
print (lv_size)

In [ ]:
for index,osd_blk in enumerate(block_dev_list):
    osd_blk_info = ods_vl_map.get(osd_blk, {})
    osd_blk_info['index'] = index
    ods_vl_map[osd_blk] = osd_blk_info
    lv_db_name = "lv_db{}".format(index)
    osd_blk_info['lv_db_name'] = lv_db_name
    osd_blk_info['vg_db_name'] = ssd_vg_name
    lv_size_perc = "{}%VG".format(lv_size)
    !ansible -i $hosts -b -m shell -a "lvcreate -l $lv_size_perc -n $lv_db_name $ssd_vg_name" ceph_node

print(json.dumps(ods_vl_map, sort_keys=True, indent=4))

In [ ]:
!ansible -i $hosts -b -m shell -a "lvdisplay" ceph_node

# HDD Data LV生成

In [ ]:
print("HDD Disk num = {}".format(len(ods_vl_map)))

In [ ]:
for key,val in ods_vl_map.items():
    index = val['index']
    osd_pv_bkl = "/dev/{}".format(key)
    osd_vg_name = "vg_data_{}".format(index)
    val['osd_vg_name'] = osd_vg_name
    osd_lv_data_name = "lv_data_{}".format(index)
    val['osd_lv_data_name'] = osd_lv_data_name
    
    !ansible -i $hosts -b -m shell -a "pvcreate $osd_pv_bkl" ceph_node
    !ansible -i $hosts -b -m shell -a "pvdisplay $osd_pv_bkl" ceph_node
    !ansible -i $hosts -b -m shell -a "vgcreate $osd_vg_name $osd_pv_bkl" ceph_node
    !ansible -i $hosts -b -m shell -a "lvcreate -l 100%FREE -n $osd_lv_data_name $osd_vg_name" ceph_node
    
print(json.dumps(ods_vl_map, sort_keys=True, indent=4))

In [ ]:
!ansible -i $hosts -b -m shell -a "vgdisplay" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "lvdisplay" ceph_node

# 作成後PV/LVM情報取得

## PV情報取得

In [ ]:
# get PV Info
ret = !ansible -i $hosts -b -m shell -a 'pvdisplay' ceph_node
pv_info_list = make_pv_info_list(ret)

pv_info_map = {}
for pv_info in pv_info_list:
    vg_name = pv_info['vg_name']
    pv_info_map[vg_name] = pv_info['pv_name']    
    
# get PV Info
ret = !ansible -i $hosts -b -m shell -a 'lvdisplay' ceph_node
lv_info_list = make_lv_info_list(ret,pv_info_map)



In [ ]:
pd.DataFrame(pv_info_list)

In [ ]:
pd.DataFrame(lv_info_list)

In [ ]:
db_pv_name = None
data_pv_name_list = []
for pv_info in pv_info_list:
    vg_name = pv_info['vg_name']
    if vg_name == 'vg_db':
        db_pv_name = pv_info['pv_name']
    elif 'vg_data_' in vg_name:
        data_pv_name_list.append(pv_info['pv_name'])
    
print("db_pv_name={}".format(db_pv_name))
print("data_pv_name_list={}".format(data_pv_name_list))


# 後始末

一時ディレクトリを削除する。

In [ ]:
!rm -fr $temp_dir